### Lesson 03
Урок 3. Парсинг данных. HTML, Beautiful Soap

Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля)

In [107]:
from pprint import pprint
from lxml import html
import requests
import time
import re

In [108]:
#Объявляем функцию, которая будет преобразовывать списки, получаемые от lxml|xpath в строки
def group(rows):
    concat = ""
    for row in rows:
        concat = concat + str(row)
        rows = concat
    return rows

In [131]:
#Задаём имя искомой вакансии
name = "архитектор"

#Задаём теоретическое максимально возможное количество страниц
max_pages = 3

#Задаём таймаут для запросов к серверу в секундах
wait = 2

page = 0
url = 'https://hh.ru/search/vacancy?text=' + name + '&page=' + str(page) + '&from=suggest_post&salary=&area=1&ored_clusters=true&enable_snippets=true'

headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}

responce = requests.get(url, headers=headers)
dom = html.fromstring(responce.text)

In [132]:
page = 0

regex = re.compile('[a-zA-Zа-яА-ЯёЁ.\s]')
#Создаём словарь, в котором ключом будет являться порядковый номер вакансии
jobdict = {}
jobnum = 1

#Проходим по всем страницам
while page < max_pages:
    url = 'https://hh.ru/search/vacancy?text=' + name + '&page=' + str(page) + '&from=suggest_post&salary=&area=1&ored_clusters=true&enable_snippets=true'
    time.sleep(wait)
    #Читаем страницу
    responce = requests.get(url, headers=headers)
    dom = html.fromstring(responce.text)
    
    #Кладём в переменную список вакансий со страницы
    joblist = dom.xpath("//h3[@class='bloko-header-section-3']/span/a/text()")
    
    #Для каждой вакансии на странице получаем информацию. N - переменная которая передаётся в качестве аргумента xpath (Поэтому с 1, а не с 0)
    n = 1
    for text in joblist:
        jobsalary = dom.xpath("(//div[@class='vacancy-serp-item-body__main-info'])[" + str(n) + "]//span[@class='bloko-header-section-3']/text()")
        #Проверяем записи в списке jobsalary   
        if not jobsalary:
            curency = None
            sal_min = None
            sal_max = None
        else:
            jobsalary = group(jobsalary)
            if "руб" in jobsalary.lower():
                curency = "рубль РФ"
            elif "usd" in jobsalary.lower():
                curency = "доллар США"
            elif "eur" in jobsalary.lower():
                curency = "Евро"
            elif "kzt" in jobsalary.lower():
                curency = "казахстанский тенге"
            else:
                curency = "other"
            
            if "от" in jobsalary.lower():
                sal_min = int(regex.sub('', jobsalary))
                sal_max = None
            elif "до" in jobsalary.lower():
                sal_max = int(regex.sub('', jobsalary))
                sal_mix = None
            elif "–" in jobsalary.lower():
                clean = regex.sub('', jobsalary)
                sal_min=int(clean[:clean.find("–")])
                sal_max=int(clean[clean.find("–")+1:])
                
        company = dom.xpath("(//div[@class='vacancy-serp-item-body__main-info'])[" + str(n) + "]//div[@class='vacancy-serp-item__meta-info-company']/a/text()")
        company = group(company)
    
        jobname = dom.xpath("(//div[@class='vacancy-serp-item-body__main-info'])[" + str(n) + "]//h3[@class='bloko-header-section-3']/span/a/text()")
        jobname = group(jobname)
        
        link = dom.xpath("(//div[@class='vacancy-serp-item-body__main-info'])[" + str(n) + "]//a[@class='serp-item__title']/@href")
        link=group(link)
        #Складываем собранную информацию в словарь
        jobdict[jobnum] = {'comp' : company, 'jobname': jobname, 'link' : link, 'sal_min' : sal_min, 'sal_max' : sal_max, 'currency' : curency}
        
        jobnum += 1
        n += 1
    
    page += 1
    #Счётчик, показывающий что программа не зависла
    print ("Обработано страниц:", page, "из", max_pages)
print("Сбор данных завершён")

Обработано страниц: 1 из 3
Обработано страниц: 2 из 3
Обработано страниц: 3 из 3
Сбор данных завершён


In [133]:
#Выводим собранный список вакансий
for num in jobdict:
    print (num, "\nКомпания:", jobdict[num]['comp'], "\nВакансия:", jobdict[num]['jobname'], "\nСсылка:", jobdict[num]['link'], "\nЗарплата min:", jobdict[num]['sal_min'], "\nЗарплата max:", jobdict[num]['sal_max'], "\nВалюта:", jobdict[num]['currency'],"\n")

1 
Компания: ООО Дежурный по Бизнесу 
Вакансия: Помощник архитектора 
Ссылка: https://hh.ru/vacancy/74118642?from=vacancy_search_list&query=%D0%B0%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D0%BE%D1%80 
Зарплата min: 15000 
Зарплата max: 50000 
Валюта: рубль РФ 

2 
Компания: ООО Тим Консалт 
Вакансия: Архитектор / Ведущий архитектор 
Ссылка: https://hh.ru/vacancy/74316335?from=vacancy_search_list&query=%D0%B0%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D0%BE%D1%80 
Зарплата min: 92000 
Зарплата max: 120000 
Валюта: рубль РФ 

3 
Компания: Прогресс 
Вакансия: Архитектор 
Ссылка: https://hh.ru/vacancy/73982346?from=vacancy_search_list&query=%D0%B0%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D0%BE%D1%80 
Зарплата min: 80000 
Зарплата max: 150000 
Валюта: рубль РФ 

4 
Компания: ООО Эда 
Вакансия: Ведущий архитектор (Архитектура интерьеров) 
Ссылка: https://hh.ru/vacancy/74266519?from=vacancy_search_list&query=%D0%B0%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D0%BE%D1%80 
Зарплата min: 143000 


In [50]:
import csv

In [51]:
headers=['comp','jobname','link','sal_min', 'sal_max', 'currency']

with open('vacancy.csv', 'w', encoding="utf-8") as f:
    writer = csv.DictWriter(f, delimiter =' ', fieldnames = headers)
    writer.writeheader()
    for n in jobdict:
        writer.writerow(jobdict[n])

In [52]:
with open('vacancy.csv', encoding='utf-8') as v:
     print(v.read())

comp jobname link sal_min sal_max currency

"Сбер. Data Science" "Python developer (Machine Learning Engineer)" https://hh.ru/vacancy/73468878?from=vacancy_search_list&query=python   

"Сбер. Data Science" "Python developer (Machine Learning Engineer)" https://hh.ru/vacancy/72074865?from=vacancy_search_list&query=python   

"«Procter & Gamble», Молодой специалист" "Дата-инженер / Data Engineer" https://hh.ru/vacancy/71236005?from=vacancy_search_list&query=python 130000  "рубль РФ"

"«Procter & Gamble», Опытный специалист" "Старший дата-инженер / Senior Data Engineer" https://hh.ru/vacancy/72161518?from=vacancy_search_list&query=python 270000  "рубль РФ"

"Сбер. IT" "Middle/Senior Data Engineer в Sber AI Lab" https://hh.ru/vacancy/72868606?from=vacancy_search_list&query=python   

"Сбер. IT" "Middle/Senior Data Engineer в Sber AI Lab" https://hh.ru/vacancy/73471713?from=vacancy_search_list&query=python   

"Сбер для экспертов" "SRE/DevOps инженер в облачную инфраструктуру SberInfra" htt

In [53]:
import pandas as pd

df = pd.read_csv('vacancy.csv', error_bad_lines = False, sep=' ', encoding='utf-8')
df.head()

C:\Users\Инна\AppData\Local\Temp\ipykernel_12444\2342352909.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('vacancy.csv', error_bad_lines = False, sep=' ', encoding='utf-8')


,comp,jobname,link,sal_min,sal_max,currency
0,Сбер. Data Science,Python developer (Machine Learning Engineer),https://hh.ru/vacancy/73468878?from=vacancy_se...,NaN,NaN,NaN
1,Сбер. Data Science,Python developer (Machine Learning Engineer),https://hh.ru/vacancy/72074865?from=vacancy_se...,NaN,NaN,NaN
2,"«Procter & Gamble», Молодой специалист",Дата-инженер / Data Engineer,https://hh.ru/vacancy/71236005?from=vacancy_se...,130000.0,NaN,рубль РФ
3,"«Procter & Gamble», Опытный специалист",Старший дата-инженер / Senior Data Engineer,https://hh.ru/vacancy/72161518?from=vacancy_se...,270000.0,NaN,рубль РФ
4,Сбер. IT,Middle/Senior Data Engineer в Sber AI Lab,https://hh.ru/vacancy/72868606?from=vacancy_se...,NaN,NaN,NaN


In [54]:
!pip3 install pymongo

Defaulting to user installation because normal site-packages is not writeable


In [74]:
from pymongo import MongoClient


In [75]:
client = MongoClient('localhost', 27017)
db = client['vacancy_hh']
collection = db['hhDB']

In [111]:
def insert_document(collection, data):
    return collection.insert_one(data).inserted_id

In [135]:
for n in jobdict:
    if collection.count_documents({ 'link': jobdict[n]['link'] }, limit=1 ) == 0:
        collection.insert_one(jobdict[n])

In [85]:
def find_job(zp):
    res=collection.find({"$or":[{ 'sal_min': { "$gte": zp } },{'sal_max': {"$gte": zp} }]})
    for x in res:
        print ("\nКомпания:", x['comp'], "\nВакансия:", x['jobname'], "\nСсылка:", x['link'], 
               "\nЗарплата min:", x['sal_min'], "\nЗарплата max:", x['sal_max'], "\nВалюта:", x['currency'],"\n")


In [86]:
zarplata = 200000
find_job(zarplata)


Компания: «Procter & Gamble», Опытный специалист 
Вакансия: Старший дата-инженер / Senior Data Engineer 
Ссылка: https://hh.ru/vacancy/72161518?from=vacancy_search_list&query=python 
Зарплата min: 270000 
Зарплата max: None 
Валюта: рубль РФ 


Компания: ООО Юнит 
Вакансия: Python Backend разработчик 
Ссылка: https://hh.ru/vacancy/73581287?from=vacancy_search_list&query=python 
Зарплата min: 130000 
Зарплата max: 200000 
Валюта: рубль РФ 


Компания: ООО Дубайт 
Вакансия: Разработчик C# / Python 
Ссылка: https://hh.ru/vacancy/71147348?from=vacancy_search_list&query=python 
Зарплата min: 300000 
Зарплата max: 500000 
Валюта: рубль РФ 


Компания: Growth.pet 
Вакансия: Разработчик Python (Web) 
Ссылка: https://hh.ru/vacancy/74015227?from=vacancy_search_list&query=python 
Зарплата min: 200000 
Зарплата max: None 
Валюта: рубль РФ 


Компания: Юрент 
Вакансия: Python разработчик 
Ссылка: https://hh.ru/vacancy/73480112?from=vacancy_search_list&query=python 
Зарплата min: 250000 
Зарплата m

Валюта: рубль РФ 


Компания: Цифровые решения, НПП 
Вакансия: Программист микроконтроллеров, Embedded software developer 
Ссылка: https://hh.ru/vacancy/72230948?from=vacancy_search_list&query=python 
Зарплата min: 200000 
Зарплата max: None 
Валюта: рубль РФ 


Компания: Сбер. IT 
Вакансия: DevOps specialist (TagMe) 
Ссылка: https://hh.ru/vacancy/70796295?from=vacancy_search_list&query=python 
Зарплата min: 300000 
Зарплата max: None 
Валюта: рубль РФ 


Компания: ФГБУ ФКП Росреестра 
Вакансия: Администратор баз данных (DBA) PostgreSQL 
Ссылка: https://hh.ru/vacancy/50465677?from=vacancy_search_list&query=python 
Зарплата min: None 
Зарплата max: 220000 
Валюта: рубль РФ 


Компания: NOTAMEDIA 
Вакансия: PHP-программист 
Ссылка: https://hh.ru/vacancy/73782800?from=vacancy_search_list&query=python 
Зарплата min: None 
Зарплата max: 250000 
Валюта: рубль РФ 


Компания: ФГБУ ФКП Росреестра 
Вакансия: Администратор СУБД Cassandra 
Ссылка: https://hh.ru/vacancy/70586945?from=vacancy_searc